In [2]:
import os
import pandas as pd
from bbox import * 
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
#for image inpainting
from skimage import data
from skimage.morphology import disk, binary_dilation
from skimage.restoration import inpaint
import xarray as xr
import numpy as np
from skimage.restoration import inpaint
import cv2
import xesmf as xe
from pykrige.ok import OrdinaryKriging
from tqdm import tqdm
#convert the NO2 units to 10^15 molec/cm^2

In [3]:
variable_name = 'vertical_column_troposphere'

###############################
# Load the datasets
###############################
full_ds_nonull = xr.open_dataset('../../data/tempo_data/inpaint_experiments/no_nulls.nc', engine='netcdf4')
unique_times, index = np.unique(full_ds_nonull['time'].values, return_index=True)
full_ds_nonull = full_ds_nonull.isel(time=index)
# full_ds_nonull = full_ds_nonull.isel(time=slice(0, 50))
full_ds_nonull[variable_name]=full_ds_nonull[variable_name]/1000000000000000

full_ds_less1 = xr.open_dataset('../../data/tempo_data/inpaint_experiments/nulls_less1perc.nc', engine='netcdf4')
unique_times, index = np.unique(full_ds_less1['time'].values, return_index=True)
full_ds_less1 = full_ds_less1.isel(time=index)
full_ds_less1[variable_name]=full_ds_less1[variable_name]/1000000000000000

variable_name = 'vertical_column_troposphere'  # Replace with the actual variable name
nan_mask = full_ds_less1[variable_name].isnull()
time_dimension_nonulls = full_ds_nonull['time']
nan_mask = nan_mask.assign_coords(time=time_dimension_nonulls)
full_ds_nonull[f"{variable_name}_masked"] = full_ds_nonull[variable_name].where(~nan_mask)

###############################
# Geos-CF
###############################
# Convert timestamps to pandas datetime
timestamps_dt = pd.to_datetime(time_dimension_nonulls)
# Get unique months
unique_months = timestamps_dt.to_period('M')

# Generate bottom-of-the-hour timestamps for each hour in the original array
bottom_of_hour = {}
for month in unique_months:
    # Filter timestamps for the current month
    month_timestamps = timestamps_dt[timestamps_dt.to_period('M') == month]
    
    # Get unique hours within the month
    unique_hours = month_timestamps.floor('h').unique()
    
    # Generate bottom-of-the-hour timestamps for these hours
    bottom_of_hour[str(month)] =  np.array(unique_hours, dtype='datetime64[ns]')
# datetime_array = np.array(datetime_list, dtype='datetime64[ns]')

times_actual = np.array(timestamps_dt, dtype='datetime64[ns]')
times_rounded = np.array(timestamps_dt.floor('h'), dtype='datetime64[ns]')
# Create the dictionary that maps rounded times to actual times
time_mapping = dict(zip(times_rounded, times_actual))

# List to store datasets
ds_list = []

# Loop through each month key
for month_key in bottom_of_hour.keys():
    datetime_array = bottom_of_hour[month_key]

    # Open the dataset for the current month
    ds = xr.open_dataset(f'../../data/geos_cf_data/geos_cf_{month_key}.nc', engine='netcdf4')

    # Select data for the current datetime_array
    ds = ds.sel(time=datetime_array)

    # Extract the 'NO2' variable
    ds_no2 = ds['TROPCOL_NO2']

    # Apply the time mapping to the 'time' coordinate
    mapped_times = [time_mapping.get(time, time) for time in ds_no2.coords['time'].values]

    # Update the 'time' coordinate with the mapped times
    ds_no2.coords['time'] = ('time', mapped_times)

    # Append to the list
    ds_list.append(ds_no2)

# Concatenate all selected datasets along the 'time' dimension
concatenated_ds = xr.concat(ds_list, dim='time')

def regrid_dataset(input_ds, ds_out, method='bilinear'):
    '''Function that takes in the data you want to regrid, as well as an xarray of the lat/long to regrid to, and returns the
    regridded data'''
    # input_ds = xr.open_dataset(input_data)
    output_ds = ds_out
    regridder = xe.Regridder(input_ds, ds_out, method)
    regridded_data = regridder(input_ds, keep_attrs=True)
    return regridded_data

ds_out = xr.Dataset(
    {
        "latitude": (["latitude"], full_ds_nonull.coords['latitude'].values),
        "longitude": (["longitude"], full_ds_nonull.coords['longitude'].values),
    }
)
concatenated_ds = regrid_dataset(concatenated_ds, ds_out)

full_ds_nonull['time'] = np.array(full_ds_nonull['time'].values, dtype='datetime64[ns]')
full_ds_nonull = xr.merge([full_ds_nonull, concatenated_ds])
full_ds_nonull[f"{variable_name}_geos_cf_fill"]=full_ds_nonull[f"{variable_name}_masked"].fillna(full_ds_nonull[f"TROPCOL_NO2"])

nan_mask['time'] = np.array(nan_mask['time'].values, dtype='datetime64[ns]')

from sklearn.metrics import mean_absolute_error, r2_score

# # Extract original values (masked by nan_mask)
original_values = full_ds_nonull.where(nan_mask)[variable_name].values
original_flat = original_values[nan_mask.values]

inpaint_values = full_ds_nonull[f"{variable_name}_geos_cf_fill"].values
inpainted_flat = inpaint_values[nan_mask.values]

mae = mean_absolute_error(original_flat, inpainted_flat)
r2 = r2_score(original_flat, inpainted_flat)

print("mae: ", mae)
print("r2: ", r2)

/opt/conda/envs/ds_env/lib/python3.11/site-packages/xesmf/smm.py:131: UserWarning: Input array is not C_CONTIGUOUS. Will affect performance.
  warnings.warn('Input array is not C_CONTIGUOUS. ' 'Will affect performance.')


mae:  1.3340624085516226
r2:  0.11332746474012623


In [4]:
variable_name = 'vertical_column_troposphere'

###############################
# Load the datasets
###############################
full_ds_nonull = xr.open_dataset('../../data/tempo_data/inpaint_experiments/no_nulls.nc', engine='netcdf4')
unique_times, index = np.unique(full_ds_nonull['time'].values, return_index=True)
full_ds_nonull = full_ds_nonull.isel(time=index)
# full_ds_nonull = full_ds_nonull.isel(time=slice(50, 100))
full_ds_nonull[variable_name]=full_ds_nonull[variable_name]/1000000000000000

full_ds_less1 = xr.open_dataset('../../data/tempo_data/inpaint_experiments/nulls_less5perc.nc', engine='netcdf4')
unique_times, index = np.unique(full_ds_less1['time'].values, return_index=True)
full_ds_less1 = full_ds_less1.isel(time=index)
full_ds_less1[variable_name]=full_ds_less1[variable_name]/1000000000000000

variable_name = 'vertical_column_troposphere'  # Replace with the actual variable name
nan_mask = full_ds_less1[variable_name].isnull()
time_dimension_nonulls = full_ds_nonull['time']
nan_mask = nan_mask.assign_coords(time=time_dimension_nonulls)
full_ds_nonull[f"{variable_name}_masked"] = full_ds_nonull[variable_name].where(~nan_mask)

###############################
# Geos-CF
###############################
# Convert timestamps to pandas datetime
timestamps_dt = pd.to_datetime(time_dimension_nonulls)
# Get unique months
unique_months = timestamps_dt.to_period('M')

# Generate bottom-of-the-hour timestamps for each hour in the original array
bottom_of_hour = {}
for month in unique_months:
    # Filter timestamps for the current month
    month_timestamps = timestamps_dt[timestamps_dt.to_period('M') == month]
    
    # Get unique hours within the month
    unique_hours = month_timestamps.floor('h')
    
    # Generate bottom-of-the-hour timestamps for these hours
    bottom_of_hour[str(month)] =  np.array(unique_hours, dtype='datetime64[ns]')
# datetime_array = np.array(datetime_list, dtype='datetime64[ns]')

times_actual = np.array(timestamps_dt, dtype='datetime64[ns]')
times_rounded = np.array(timestamps_dt.floor('h'), dtype='datetime64[ns]')
# Create the dictionary that maps rounded times to actual times
time_mapping = dict(zip(times_rounded, times_actual))

# List to store datasets
ds_list = []

# Loop through each month key
for month_key in bottom_of_hour.keys():
    datetime_array = bottom_of_hour[month_key]

    # Open the dataset for the current month
    ds = xr.open_dataset(f'../../data/geos_cf_data/geos_cf_{month_key}.nc', engine='netcdf4')

    # Select data for the current datetime_array
    ds = ds.sel(time=datetime_array)

    # Extract the 'NO2' variable
    ds_no2 = ds['TROPCOL_NO2']

    # Apply the time mapping to the 'time' coordinate
    # mapped_times = [time_mapping.get(time, time) for time in ds_no2.coords['time'].values]

    # Update the 'time' coordinate with the mapped times
    # ds_no2.coords['time'] = ('time', mapped_times)

    # Append to the list
    ds_list.append(ds_no2)

# Concatenate all selected datasets along the 'time' dimension
concatenated_ds = xr.concat(ds_list, dim='time')
concatenated_ds = concatenated_ds.assign_coords(time=sorted(times_actual))

def regrid_dataset(input_ds, ds_out, method='bilinear'):
    '''Function that takes in the data you want to regrid, as well as an xarray of the lat/long to regrid to, and returns the
    regridded data'''
    # input_ds = xr.open_dataset(input_data)
    output_ds = ds_out
    regridder = xe.Regridder(input_ds, ds_out, method)
    regridded_data = regridder(input_ds, keep_attrs=True)
    return regridded_data

ds_out = xr.Dataset(
    {
        "latitude": (["latitude"], full_ds_nonull.coords['latitude'].values),
        "longitude": (["longitude"], full_ds_nonull.coords['longitude'].values),
    }
)
concatenated_ds = regrid_dataset(concatenated_ds, ds_out)

full_ds_nonull['time'] = np.array(full_ds_nonull['time'].values, dtype='datetime64[ns]')
full_ds_nonull = xr.merge([full_ds_nonull, concatenated_ds])
full_ds_nonull[f"{variable_name}_geos_cf_fill"]=full_ds_nonull[f"{variable_name}_masked"].fillna(full_ds_nonull[f"TROPCOL_NO2"])

nan_mask['time'] = np.array(nan_mask['time'].values, dtype='datetime64[ns]')

from sklearn.metrics import mean_absolute_error, r2_score

# # Extract original values (masked by nan_mask)
original_values = full_ds_nonull.where(nan_mask)[variable_name].values
original_flat = original_values[nan_mask.values]

inpaint_values = full_ds_nonull[f"{variable_name}_geos_cf_fill"].values
inpainted_flat = inpaint_values[nan_mask.values]

mae = mean_absolute_error(original_flat, inpainted_flat)
r2 = r2_score(original_flat, inpainted_flat)

print("mae: ", mae)
print("r2: ", r2)

/opt/conda/envs/ds_env/lib/python3.11/site-packages/xesmf/smm.py:131: UserWarning: Input array is not C_CONTIGUOUS. Will affect performance.
  warnings.warn('Input array is not C_CONTIGUOUS. ' 'Will affect performance.')


mae:  1.4510752586758724
r2:  0.0524055732950105


In [5]:
variable_name = 'vertical_column_troposphere'

###############################
# Load the datasets
###############################
full_ds_nonull = xr.open_dataset('../../data/tempo_data/inpaint_experiments/no_nulls.nc', engine='netcdf4')
unique_times, index = np.unique(full_ds_nonull['time'].values, return_index=True)
full_ds_nonull = full_ds_nonull.isel(time=index)
# full_ds_nonull = full_ds_nonull.isel(time=slice(100, 150))
full_ds_nonull[variable_name]=full_ds_nonull[variable_name]/1000000000000000

full_ds_less1 = xr.open_dataset('../../data/tempo_data/inpaint_experiments/nulls_less10perc.nc', engine='netcdf4')
unique_times, index = np.unique(full_ds_less1['time'].values, return_index=True)
full_ds_less1 = full_ds_less1.isel(time=index)
full_ds_less1[variable_name]=full_ds_less1[variable_name]/1000000000000000

variable_name = 'vertical_column_troposphere'  # Replace with the actual variable name
nan_mask = full_ds_less1[variable_name].isnull()
time_dimension_nonulls = full_ds_nonull['time']
nan_mask = nan_mask.assign_coords(time=time_dimension_nonulls)
full_ds_nonull[f"{variable_name}_masked"] = full_ds_nonull[variable_name].where(~nan_mask)

###############################
# Geos-CF
###############################
# Convert timestamps to pandas datetime
timestamps_dt = pd.to_datetime(time_dimension_nonulls)
# Get unique months
unique_months = timestamps_dt.to_period('M')

# Generate bottom-of-the-hour timestamps for each hour in the original array
bottom_of_hour = {}
for month in unique_months:
    # Filter timestamps for the current month
    month_timestamps = timestamps_dt[timestamps_dt.to_period('M') == month]
    
    # Get unique hours within the month
    unique_hours = month_timestamps.floor('h')
    
    # Generate bottom-of-the-hour timestamps for these hours
    bottom_of_hour[str(month)] =  np.array(unique_hours, dtype='datetime64[ns]')
# datetime_array = np.array(datetime_list, dtype='datetime64[ns]')

times_actual = np.array(timestamps_dt, dtype='datetime64[ns]')
times_rounded = np.array(timestamps_dt.floor('h'), dtype='datetime64[ns]')
# Create the dictionary that maps rounded times to actual times
time_mapping = dict(zip(times_rounded, times_actual))

# List to store datasets
ds_list = []

# Loop through each month key
for month_key in bottom_of_hour.keys():
    datetime_array = bottom_of_hour[month_key]

    # Open the dataset for the current month
    ds = xr.open_dataset(f'../../data/geos_cf_data/geos_cf_{month_key}.nc', engine='netcdf4')

    # Select data for the current datetime_array
    ds = ds.sel(time=datetime_array)

    # Extract the 'NO2' variable
    ds_no2 = ds['TROPCOL_NO2']

    # Apply the time mapping to the 'time' coordinate
    # mapped_times = [time_mapping.get(time, time) for time in ds_no2.coords['time'].values]

    # Update the 'time' coordinate with the mapped times
    # ds_no2.coords['time'] = ('time', mapped_times)

    # Append to the list
    ds_list.append(ds_no2)

# Concatenate all selected datasets along the 'time' dimension
concatenated_ds = xr.concat(ds_list, dim='time')
concatenated_ds = concatenated_ds.assign_coords(time=sorted(times_actual))

def regrid_dataset(input_ds, ds_out, method='bilinear'):
    '''Function that takes in the data you want to regrid, as well as an xarray of the lat/long to regrid to, and returns the
    regridded data'''
    # input_ds = xr.open_dataset(input_data)
    output_ds = ds_out
    regridder = xe.Regridder(input_ds, ds_out, method)
    regridded_data = regridder(input_ds, keep_attrs=True)
    return regridded_data

ds_out = xr.Dataset(
    {
        "latitude": (["latitude"], full_ds_nonull.coords['latitude'].values),
        "longitude": (["longitude"], full_ds_nonull.coords['longitude'].values),
    }
)
concatenated_ds = regrid_dataset(concatenated_ds, ds_out)

full_ds_nonull['time'] = np.array(full_ds_nonull['time'].values, dtype='datetime64[ns]')
full_ds_nonull = xr.merge([full_ds_nonull, concatenated_ds])
full_ds_nonull[f"{variable_name}_geos_cf_fill"]=full_ds_nonull[f"{variable_name}_masked"].fillna(full_ds_nonull[f"TROPCOL_NO2"])

nan_mask['time'] = np.array(nan_mask['time'].values, dtype='datetime64[ns]')

from sklearn.metrics import mean_absolute_error, r2_score

# # Extract original values (masked by nan_mask)
original_values = full_ds_nonull.where(nan_mask)[variable_name].values
original_flat = original_values[nan_mask.values]

inpaint_values = full_ds_nonull[f"{variable_name}_geos_cf_fill"].values
inpainted_flat = inpaint_values[nan_mask.values]

mae = mean_absolute_error(original_flat, inpainted_flat)
r2 = r2_score(original_flat, inpainted_flat)

print("mae: ", mae)
print("r2: ", r2)

/opt/conda/envs/ds_env/lib/python3.11/site-packages/xesmf/smm.py:131: UserWarning: Input array is not C_CONTIGUOUS. Will affect performance.
  warnings.warn('Input array is not C_CONTIGUOUS. ' 'Will affect performance.')


mae:  1.3630930523120253
r2:  0.09510180531829182


In [6]:
variable_name = 'vertical_column_troposphere'

###############################
# Load the datasets
###############################
full_ds_nonull = xr.open_dataset('../../data/tempo_data/inpaint_experiments/no_nulls.nc', engine='netcdf4')
unique_times, index = np.unique(full_ds_nonull['time'].values, return_index=True)
full_ds_nonull = full_ds_nonull.isel(time=index)
# full_ds_nonull = full_ds_nonull.isel(time=slice(100, 150))
full_ds_nonull[variable_name]=full_ds_nonull[variable_name]/1000000000000000

full_ds_less1 = xr.open_dataset('../../data/tempo_data/inpaint_experiments/nulls_less25perc.nc', engine='netcdf4')
unique_times, index = np.unique(full_ds_less1['time'].values, return_index=True)
full_ds_less1 = full_ds_less1.isel(time=index)
full_ds_less1[variable_name]=full_ds_less1[variable_name]/1000000000000000

variable_name = 'vertical_column_troposphere'  # Replace with the actual variable name
nan_mask = full_ds_less1[variable_name].isnull()
time_dimension_nonulls = full_ds_nonull['time']
nan_mask = nan_mask.assign_coords(time=time_dimension_nonulls)
full_ds_nonull[f"{variable_name}_masked"] = full_ds_nonull[variable_name].where(~nan_mask)

###############################
# Geos-CF
###############################
# Convert timestamps to pandas datetime
timestamps_dt = pd.to_datetime(time_dimension_nonulls)
# Get unique months
unique_months = timestamps_dt.to_period('M')

# Generate bottom-of-the-hour timestamps for each hour in the original array
bottom_of_hour = {}
for month in unique_months:
    # Filter timestamps for the current month
    month_timestamps = timestamps_dt[timestamps_dt.to_period('M') == month]
    
    # Get unique hours within the month
    unique_hours = month_timestamps.floor('h')
    
    # Generate bottom-of-the-hour timestamps for these hours
    bottom_of_hour[str(month)] =  np.array(unique_hours, dtype='datetime64[ns]')
# datetime_array = np.array(datetime_list, dtype='datetime64[ns]')

times_actual = np.array(timestamps_dt, dtype='datetime64[ns]')
times_rounded = np.array(timestamps_dt.floor('h'), dtype='datetime64[ns]')
# Create the dictionary that maps rounded times to actual times
time_mapping = dict(zip(times_rounded, times_actual))

# List to store datasets
ds_list = []

# Loop through each month key
for month_key in bottom_of_hour.keys():
    datetime_array = bottom_of_hour[month_key]

    # Open the dataset for the current month
    ds = xr.open_dataset(f'../../data/geos_cf_data/geos_cf_{month_key}.nc', engine='netcdf4')

    # Select data for the current datetime_array
    ds = ds.sel(time=datetime_array)

    # Extract the 'NO2' variable
    ds_no2 = ds['TROPCOL_NO2']

    # Apply the time mapping to the 'time' coordinate
    # mapped_times = [time_mapping.get(time, time) for time in ds_no2.coords['time'].values]

    # Update the 'time' coordinate with the mapped times
    # ds_no2.coords['time'] = ('time', mapped_times)

    # Append to the list
    ds_list.append(ds_no2)

# Concatenate all selected datasets along the 'time' dimension
concatenated_ds = xr.concat(ds_list, dim='time')
concatenated_ds = concatenated_ds.assign_coords(time=sorted(times_actual))

def regrid_dataset(input_ds, ds_out, method='bilinear'):
    '''Function that takes in the data you want to regrid, as well as an xarray of the lat/long to regrid to, and returns the
    regridded data'''
    # input_ds = xr.open_dataset(input_data)
    output_ds = ds_out
    regridder = xe.Regridder(input_ds, ds_out, method)
    regridded_data = regridder(input_ds, keep_attrs=True)
    return regridded_data

ds_out = xr.Dataset(
    {
        "latitude": (["latitude"], full_ds_nonull.coords['latitude'].values),
        "longitude": (["longitude"], full_ds_nonull.coords['longitude'].values),
    }
)
concatenated_ds = regrid_dataset(concatenated_ds, ds_out)

full_ds_nonull['time'] = np.array(full_ds_nonull['time'].values, dtype='datetime64[ns]')
full_ds_nonull = xr.merge([full_ds_nonull, concatenated_ds])
full_ds_nonull[f"{variable_name}_geos_cf_fill"]=full_ds_nonull[f"{variable_name}_masked"].fillna(full_ds_nonull[f"TROPCOL_NO2"])

nan_mask['time'] = np.array(nan_mask['time'].values, dtype='datetime64[ns]')

from sklearn.metrics import mean_absolute_error, r2_score

# # Extract original values (masked by nan_mask)
original_values = full_ds_nonull.where(nan_mask)[variable_name].values
original_flat = original_values[nan_mask.values]

inpaint_values = full_ds_nonull[f"{variable_name}_geos_cf_fill"].values
inpainted_flat = inpaint_values[nan_mask.values]

mae = mean_absolute_error(original_flat, inpainted_flat)
r2 = r2_score(original_flat, inpainted_flat)

print("mae: ", mae)
print("r2: ", r2)

/opt/conda/envs/ds_env/lib/python3.11/site-packages/xesmf/smm.py:131: UserWarning: Input array is not C_CONTIGUOUS. Will affect performance.
  warnings.warn('Input array is not C_CONTIGUOUS. ' 'Will affect performance.')


mae:  1.4860333766451692
r2:  0.07266527960822511


In [7]:
variable_name = 'vertical_column_troposphere'

###############################
# Load the datasets
###############################
full_ds_nonull = xr.open_dataset('../../data/tempo_data/inpaint_experiments/no_nulls.nc', engine='netcdf4')
unique_times, index = np.unique(full_ds_nonull['time'].values, return_index=True)
full_ds_nonull = full_ds_nonull.isel(time=index)
# full_ds_nonull = full_ds_nonull.isel(time=slice(150, 200))
full_ds_nonull[variable_name]=full_ds_nonull[variable_name]/1000000000000000

full_ds_less1 = xr.open_dataset('../../data/tempo_data/inpaint_experiments/nulls_less50perc.nc', engine='netcdf4')
unique_times, index = np.unique(full_ds_less1['time'].values, return_index=True)
full_ds_less1 = full_ds_less1.isel(time=index)
full_ds_less1[variable_name]=full_ds_less1[variable_name]/1000000000000000

variable_name = 'vertical_column_troposphere'  # Replace with the actual variable name
nan_mask = full_ds_less1[variable_name].isnull()
time_dimension_nonulls = full_ds_nonull['time']
nan_mask = nan_mask.assign_coords(time=time_dimension_nonulls)
full_ds_nonull[f"{variable_name}_masked"] = full_ds_nonull[variable_name].where(~nan_mask)

###############################
# Geos-CF
###############################
# Convert timestamps to pandas datetime
timestamps_dt = pd.to_datetime(time_dimension_nonulls)
# Get unique months
unique_months = timestamps_dt.to_period('M')

# Generate bottom-of-the-hour timestamps for each hour in the original array
bottom_of_hour = {}
for month in unique_months:
    # Filter timestamps for the current month
    month_timestamps = timestamps_dt[timestamps_dt.to_period('M') == month]
    
    # Get unique hours within the month
    unique_hours = month_timestamps.floor('h')
    
    # Generate bottom-of-the-hour timestamps for these hours
    bottom_of_hour[str(month)] =  np.array(unique_hours, dtype='datetime64[ns]')
# datetime_array = np.array(datetime_list, dtype='datetime64[ns]')

times_actual = np.array(timestamps_dt, dtype='datetime64[ns]')
times_rounded = np.array(timestamps_dt.floor('h'), dtype='datetime64[ns]')
# Create the dictionary that maps rounded times to actual times
time_mapping = dict(zip(times_rounded, times_actual))

# List to store datasets
ds_list = []

# Loop through each month key
for month_key in bottom_of_hour.keys():
    datetime_array = bottom_of_hour[month_key]

    # Open the dataset for the current month
    ds = xr.open_dataset(f'../../data/geos_cf_data/geos_cf_{month_key}.nc', engine='netcdf4')

    # Select data for the current datetime_array
    ds = ds.sel(time=datetime_array)

    # Extract the 'NO2' variable
    ds_no2 = ds['TROPCOL_NO2']

    # Apply the time mapping to the 'time' coordinate
    # mapped_times = [time_mapping.get(time, time) for time in ds_no2.coords['time'].values]

    # Update the 'time' coordinate with the mapped times
    # ds_no2.coords['time'] = ('time', mapped_times)

    # Append to the list
    ds_list.append(ds_no2)

# Concatenate all selected datasets along the 'time' dimension
concatenated_ds = xr.concat(ds_list, dim='time')
concatenated_ds = concatenated_ds.assign_coords(time=sorted(times_actual))

def regrid_dataset(input_ds, ds_out, method='bilinear'):
    '''Function that takes in the data you want to regrid, as well as an xarray of the lat/long to regrid to, and returns the
    regridded data'''
    # input_ds = xr.open_dataset(input_data)
    output_ds = ds_out
    regridder = xe.Regridder(input_ds, ds_out, method)
    regridded_data = regridder(input_ds, keep_attrs=True)
    return regridded_data

ds_out = xr.Dataset(
    {
        "latitude": (["latitude"], full_ds_nonull.coords['latitude'].values),
        "longitude": (["longitude"], full_ds_nonull.coords['longitude'].values),
    }
)
concatenated_ds = regrid_dataset(concatenated_ds, ds_out)

full_ds_nonull['time'] = np.array(full_ds_nonull['time'].values, dtype='datetime64[ns]')
full_ds_nonull = xr.merge([full_ds_nonull, concatenated_ds])
full_ds_nonull[f"{variable_name}_geos_cf_fill"]=full_ds_nonull[f"{variable_name}_masked"].fillna(full_ds_nonull[f"TROPCOL_NO2"])

nan_mask['time'] = np.array(nan_mask['time'].values, dtype='datetime64[ns]')

from sklearn.metrics import mean_absolute_error, r2_score

# # Extract original values (masked by nan_mask)
original_values = full_ds_nonull.where(nan_mask)[variable_name].values
original_flat = original_values[nan_mask.values]

inpaint_values = full_ds_nonull[f"{variable_name}_geos_cf_fill"].values
inpainted_flat = inpaint_values[nan_mask.values]

mae = mean_absolute_error(original_flat, inpainted_flat)
r2 = r2_score(original_flat, inpainted_flat)

print("mae: ", mae)
print("r2: ", r2)

/opt/conda/envs/ds_env/lib/python3.11/site-packages/xesmf/smm.py:131: UserWarning: Input array is not C_CONTIGUOUS. Will affect performance.
  warnings.warn('Input array is not C_CONTIGUOUS. ' 'Will affect performance.')


mae:  1.4058511680447383
r2:  -0.006789968653210687


In [8]:
variable_name = 'vertical_column_troposphere'

###############################
# Load the datasets
###############################
full_ds_nonull = xr.open_dataset('../../data/tempo_data/inpaint_experiments/no_nulls.nc', engine='netcdf4')
unique_times, index = np.unique(full_ds_nonull['time'].values, return_index=True)
full_ds_nonull = full_ds_nonull.isel(time=index)
# full_ds_nonull = full_ds_nonull.isel(time=slice(150, 200))
full_ds_nonull[variable_name]=full_ds_nonull[variable_name]/1000000000000000

full_ds_less1 = xr.open_dataset('../../data/tempo_data/inpaint_experiments/nulls_over50perc.nc', engine='netcdf4')
unique_times, index = np.unique(full_ds_less1['time'].values, return_index=True)
full_ds_less1 = full_ds_less1.isel(time=index)
full_ds_less1[variable_name]=full_ds_less1[variable_name]/1000000000000000

variable_name = 'vertical_column_troposphere'  # Replace with the actual variable name
nan_mask = full_ds_less1[variable_name].isnull()
time_dimension_nonulls = full_ds_nonull['time']
nan_mask = nan_mask.assign_coords(time=time_dimension_nonulls)
full_ds_nonull[f"{variable_name}_masked"] = full_ds_nonull[variable_name].where(~nan_mask)

###############################
# Geos-CF
###############################
# Convert timestamps to pandas datetime
timestamps_dt = pd.to_datetime(time_dimension_nonulls)
# Get unique months
unique_months = timestamps_dt.to_period('M')

# Generate bottom-of-the-hour timestamps for each hour in the original array
bottom_of_hour = {}
for month in unique_months:
    # Filter timestamps for the current month
    month_timestamps = timestamps_dt[timestamps_dt.to_period('M') == month]
    
    # Get unique hours within the month
    unique_hours = month_timestamps.floor('h')
    
    # Generate bottom-of-the-hour timestamps for these hours
    bottom_of_hour[str(month)] =  np.array(unique_hours, dtype='datetime64[ns]')
# datetime_array = np.array(datetime_list, dtype='datetime64[ns]')

times_actual = np.array(timestamps_dt, dtype='datetime64[ns]')
times_rounded = np.array(timestamps_dt.floor('h'), dtype='datetime64[ns]')
# Create the dictionary that maps rounded times to actual times
time_mapping = dict(zip(times_rounded, times_actual))

# List to store datasets
ds_list = []

# Loop through each month key
for month_key in bottom_of_hour.keys():
    datetime_array = bottom_of_hour[month_key]

    # Open the dataset for the current month
    ds = xr.open_dataset(f'../../data/geos_cf_data/geos_cf_{month_key}.nc', engine='netcdf4')

    # Select data for the current datetime_array
    ds = ds.sel(time=datetime_array)

    # Extract the 'NO2' variable
    ds_no2 = ds['TROPCOL_NO2']

    # Apply the time mapping to the 'time' coordinate
    # mapped_times = [time_mapping.get(time, time) for time in ds_no2.coords['time'].values]

    # Update the 'time' coordinate with the mapped times
    # ds_no2.coords['time'] = ('time', mapped_times)

    # Append to the list
    ds_list.append(ds_no2)

# Concatenate all selected datasets along the 'time' dimension
concatenated_ds = xr.concat(ds_list, dim='time')
concatenated_ds = concatenated_ds.assign_coords(time=sorted(times_actual))

def regrid_dataset(input_ds, ds_out, method='bilinear'):
    '''Function that takes in the data you want to regrid, as well as an xarray of the lat/long to regrid to, and returns the
    regridded data'''
    # input_ds = xr.open_dataset(input_data)
    output_ds = ds_out
    regridder = xe.Regridder(input_ds, ds_out, method)
    regridded_data = regridder(input_ds, keep_attrs=True)
    return regridded_data

ds_out = xr.Dataset(
    {
        "latitude": (["latitude"], full_ds_nonull.coords['latitude'].values),
        "longitude": (["longitude"], full_ds_nonull.coords['longitude'].values),
    }
)
concatenated_ds = regrid_dataset(concatenated_ds, ds_out)

full_ds_nonull['time'] = np.array(full_ds_nonull['time'].values, dtype='datetime64[ns]')
full_ds_nonull = xr.merge([full_ds_nonull, concatenated_ds])
full_ds_nonull[f"{variable_name}_geos_cf_fill"]=full_ds_nonull[f"{variable_name}_masked"].fillna(full_ds_nonull[f"TROPCOL_NO2"])

nan_mask['time'] = np.array(nan_mask['time'].values, dtype='datetime64[ns]')

from sklearn.metrics import mean_absolute_error, r2_score

# # Extract original values (masked by nan_mask)
original_values = full_ds_nonull.where(nan_mask)[variable_name].values
original_flat = original_values[nan_mask.values]

inpaint_values = full_ds_nonull[f"{variable_name}_geos_cf_fill"].values
inpainted_flat = inpaint_values[nan_mask.values]

mae = mean_absolute_error(original_flat, inpainted_flat)
r2 = r2_score(original_flat, inpainted_flat)

print("mae: ", mae)
print("r2: ", r2)

/opt/conda/envs/ds_env/lib/python3.11/site-packages/xesmf/smm.py:131: UserWarning: Input array is not C_CONTIGUOUS. Will affect performance.
  warnings.warn('Input array is not C_CONTIGUOUS. ' 'Will affect performance.')


mae:  1.4833248088567077
r2:  0.06481171975288813


In [12]:
full_ds_nonull[f"{variable_name}_geos_cf_fill"].count()

<xarray.DataArray 'vertical_column_troposphere_geos_cf_fill' ()> Size: 8B
array(12720000)

In [ ]:
import matplotlib.pyplot as plt
# # Define your latitude and longitude bounds
# lat_min, lat_max = 28.6, 33.4  # Example latitude range
# lon_min, lon_max = -98.9, -88.3  # Example longitude range

# Select one hour of temperature data (e.g., the first timestamp)
hour_index = 0 #Change this to select a different hour if desired
temperature_data = full_ds_nonull[f"{variable_name}_geos_cf_fill"].isel(time=hour_index)
# temperature_data = ds['Percent_Tree_Cover']

# Plot the data with switched axes
plt.figure(figsize=(10, 6))

# Plot with latitude on x-axis and longitude on y-axis
temperature_data.T.plot(
    cmap="coolwarm",  # Colormap for temperature visualization
    cbar_kwargs={'label': 'Temperature (K)'}  # Add color bar label
)

# Update axis labels
plt.xlabel("Latitude")
plt.ylabel("Longitude")

plt.show()

In [13]:
mae

1.5685458006039108

In [14]:
r2

0.2284887116882628